# As mulheres perderam mais renda do trabalho durante a pandemia? Uma análise em painel usando a PNAD COVID para o Sudeste

----

## Leitura dos Dados

In [ ]:
## Importando o que for necessário
import pandas as pd
import numpy as np
from scipy import stats
from zipfile import ZipFile
import statsmodels.api as sm
import statsmodels.stats.api as sms
from IPython.display import clear_output # limpa o output de uma célula

In [ ]:
## Modelos de painel
#! pip install linearmodels  # já atualiza o statsmodels (caso precise)
from linearmodels import PooledOLS, PanelOLS, RandomEffects
from linearmodels.panel import compare

## Para funções com modelos de painel, ver:
# https://bashtage.github.io/linearmodels/panel/introduction.html

In [ ]:
## Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Lendo todos os dados de maio a novembro
sCaminho = "/content/drive/Othercomputers/Meu modelo MacBook Air/Documents/IPEA/Curso IDP/Dados/"

# Lista contendo todos os DataFrames
lDataFrames = []

# Loop para ler cada mês (maio a novembro) e adicionando cada base de cada mês a uma lista
for i in range(5, 12):
  sMes = str(i).zfill(2)  # colocando 0, caso precise
  with ZipFile(sCaminho + 'PNAD_COVID_'+sMes+'2020.zip') as z:
    with z.open('PNAD_COVID_'+sMes+'2020.csv') as f:
      lDataFrames.append(pd.read_csv(f))

## Criando um DataFrame com todos os dados usando o concat
PNAD_COVID = pd.concat(lDataFrames)

# Vendo o tamanho da base
print(f"Linhas: {PNAD_COVID.shape[0]}; Colunas: {PNAD_COVID.shape[1]}")

Linhas: 2650459; Colunas: 148


In [ ]:
## Vendo observações por mês
PNAD_COVID.groupby('V1013').size()

V1013
5     349306
6     381270
7     384166
8     386520
9     387298
10    380461
11    381438
dtype: int64

## Taxa de Desocupação, Taxa de Participação e Nível de Ocupação

- A **taxa de desocupação** é calculada em relação à **força de trabalho**, ou seja, pessoas que trabalham ou estão procurando emprego.

- A **taxa de participação** é calculada como a força de trabalho frente à população total.

- O **nível de ocupação** é calculado como a razão da população ocupada frente à total.

In [ ]:
## Queremos determinar se uma pessoa é ocupada/desocupada
## Ocupado
PNAD_COVID["Ocupado"] = PNAD_COVID["C007"].apply(lambda i: 1 if i <= 8 else np.nan)

## Desocupado
PNAD_COVID["Desocupado"] = PNAD_COVID["C015"].apply(lambda i: 1 if i == 1 else np.nan)

## Fora da Força de Trabalho
PNAD_COVID["Fora_FT"] = PNAD_COVID["C015"].apply(lambda i: 1 if i == 2 else np.nan)

In [ ]:
## Vendo Ocupados e Desocupados por Mês (sem pesos)
# V1013: mês!
dfOcupacao = pd.DataFrame(PNAD_COVID.groupby("V1013")[["Ocupado","Desocupado","Fora_FT", "V1032"]].count())
dfOcupacao["Taxa_Desocupacao"] = dfOcupacao["Desocupado"] / (dfOcupacao["Ocupado"] + dfOcupacao["Desocupado"])
dfOcupacao["Taxa_Participacao"] = (dfOcupacao["Ocupado"] + dfOcupacao["Desocupado"]) / dfOcupacao["V1032"]
dfOcupacao["Nivel_Ocupacao"] = dfOcupacao["Ocupado"] / dfOcupacao["V1032"]
dfOcupacao[["Taxa_Desocupacao", "Taxa_Participacao", "Nivel_Ocupacao"]]*100

,Taxa_Desocupacao,Taxa_Participacao,Nivel_Ocupacao
V1013,,,
5,9.937047,43.338219,39.031680
6,11.366228,43.236552,38.322186
7,11.969057,42.230442,37.175856
8,12.472072,42.614095,37.299234
9,12.938650,43.168051,37.582688
10,13.020968,43.710393,38.018877
11,13.301538,44.086064,38.221939


### Desafio: como calcular com pesos?

Resultados do IBGE: https://agenciabrasil.ebc.com.br/economia/noticia/2020-12/pnad-covid-19-desemprego-chega-142-em-novembro

In [ ]:
## Calculando variáveis com pesos por mês
# V1032: peso!
ocupados_mes = PNAD_COVID.groupby(["V1013", "Ocupado"])["V1032"].sum()
desocupados_mes = PNAD_COVID.groupby(["V1013", "Desocupado"])["V1032"].sum()
pia_mes = PNAD_COVID.query("A002 >= 14").groupby(["V1013", "Ano"])["V1032"].sum()
# PIA: população em idade ativa

In [ ]:
## Número de Desocupados (milhões)
desocupados_mes / 1000000

V1013  Desocupado
5      1.0           10.129077
6      1.0           11.815274
7      1.0           12.253093
8      1.0           12.926400
9      1.0           13.486289
10     1.0           13.763069
11     1.0           14.038444
Name: V1032, dtype: float64

In [ ]:
## Taxa de Desocupação
100 * desocupados_mes / (desocupados_mes + ocupados_mes)

V1013  Desocupado
5      1.0           10.714827
6      1.0           12.402611
7      1.0           13.071819
8      1.0           13.597067
9      1.0           13.986948
10     1.0           14.058701
11     1.0           14.223454
Name: V1032, dtype: float64

In [ ]:
## Taxa de Participação
100 * (desocupados_mes + ocupados_mes) / pia_mes

V1013  Desocupado
5      1.0           55.642742
6      1.0           55.989828
7      1.0           55.071697
8      1.0           55.819533
9      1.0           56.541493
10     1.0           57.383688
11     1.0           57.806241
Name: V1032, dtype: float64

In [ ]:
## Nível de Ocupação
100 * ocupados_mes / pia_mes

V1013  Ocupado
5      1.0        49.680718
6      1.0        49.045627
7      1.0        47.872824
8      1.0        48.229714
9      1.0        48.633064
10     1.0        49.316287
11     1.0        49.584198
Name: V1032, dtype: float64

## Criando o Painel

In [ ]:
## Filtrando apenas para o Sudeste
PNAD_COVID = PNAD_COVID.loc[PNAD_COVID["UF"].isin([31, 32, 33, 35])]

In [ ]:
# Vendo o tamanho da base
print(f"Linhas: {PNAD_COVID.shape[0]}; Colunas: {PNAD_COVID.shape[1]}")

Linhas: 781890; Colunas: 151


In [ ]:
## Vendo observações por mês com a base filtrada
PNAD_COVID.groupby('V1013').size()

V1013
5     105074
6     113335
7     113801
8     113647
9     113135
10    111527
11    111371
dtype: int64

### Identificando e Balanceado o *Dataset*

In [ ]:
## Criando id do domicílio e do indivíduo
# Operador '+' com strings: concatenação
PNAD_COVID['iddom'] = PNAD_COVID["UPA"].astype(str) + PNAD_COVID["V1008"].astype(str)
PNAD_COVID['idind'] = PNAD_COVID['iddom'].astype(str) + PNAD_COVID['A001'].astype(str) + \
  PNAD_COVID['A001B1'].astype(str) + PNAD_COVID['A001B2'].astype(str) + PNAD_COVID['A001B3'].astype(str) # data de aniversário

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
## Contando quantos indivíduos tem
nIndividuos = PNAD_COVID['idind'].unique().size
nIndividuos

141175

In [ ]:
PNAD_COVID[["iddom","idind"]]

,iddom,idind
145694,3101405851,3101405851125111968
145695,3101405851,3101405851229121969
145696,3101405852,310140585212711991
145697,3101405853,310140585311151982
145698,3101405853,310140585323131984
...,...,...
275192,35028078113,35028078113214111980
275193,35028078113,3502807811332822001
275194,35028078114,3502807811411561970
275195,35028078114,35028078114226111972


In [ ]:
## Ordenando o DataFrame por id do indivíduo e mês (V1013)
PNAD_COVID.sort_values(by=['idind', 'V1013'], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
## Vendo o número de aparições de cada indivíduo e adicionando como coluna usando transform
PNAD_COVID["Aparições"] = PNAD_COVID.groupby('idind')['idind'].transform('count')
PNAD_COVID[["idind", "Aparições"]].head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,idind,Aparições
158183,3100005091012331984,6
175367,3100005091012331984,6
179282,3100005091012331984,6
180200,3100005091012331984,6
176455,3100005091012331984,6
177556,3100005091012331984,6
158184,3100005091022692004,6
175368,3100005091022692004,6
179283,3100005091022692004,6
180201,3100005091022692004,6


In [ ]:
## Vendo distribuição de aparições (xtdescribe)
PNAD_COVID.drop_duplicates(subset=['idind'], keep='first')["Aparições"].value_counts(normalize=True)

7    0.509814
6    0.153136
5    0.085093
1    0.070189
4    0.066974
3    0.057794
2    0.057000
Name: Aparições, dtype: float64

In [ ]:
## Pegando apenas as pessoas com pelo menos 5 entrevistas
# Evita problemas que surgem com paineis muito desbalanceados
PNAD_COVID = PNAD_COVID.query("Aparições >= 5")

In [ ]:
## Vendo o novo tamanho da base
print(f"Linhas: {PNAD_COVID.shape[0]}; Colunas: {PNAD_COVID.shape[1]}")

Linhas: 693590; Colunas: 154


### Montando a Estrutura do Painel

- Primeiro Nível: idind (indivíduo)
- Segundo Nível: mês/V1013 (tempo)

In [ ]:
## Renomeando coluna de mês
PNAD_COVID.rename(columns={"V1013":"Mês"}, inplace=True)

In [ ]:
## Montando a estrutura do painel
PNAD_COVID = PNAD_COVID.set_index(["idind", "Mês"], drop=False)

In [ ]:
## Vendo o resultado
PNAD_COVID.head(20)

Ano  UF  ...                idind  Aparições
idind               Mês            ...                                
3100005091012331984 5    2020  31  ...  3100005091012331984          6
                    6    2020  31  ...  3100005091012331984          6
                    8    2020  31  ...  3100005091012331984          6
                    9    2020  31  ...  3100005091012331984          6
                    10   2020  31  ...  3100005091012331984          6
                    11   2020  31  ...  3100005091012331984          6
3100005091022692004 5    2020  31  ...  3100005091022692004          6
                    6    2020  31  ...  3100005091022692004          6
                    8    2020  31  ...  3100005091022692004          6
                    9    2020  31  ...  3100005091022692004          6
                    10   2020  31  ...  3100005091022692004          6
                    11   2020  31  ...  3100005091022692004          6
3100005091112341954 5    2020  31  ...  3100005091112341954          7
                    6    2020  31  ...  3100005091112341954          7
                    7    2020  31  ...  3100005091112341954          7
                    8    2020  31  ...  3100005091112341954          7
                    9    2020  31  ...  3100005091112341954          7
                    10   2020  31  ...  3100005091112341954          7
                    11   2020  31  ...  3100005091112341954          7
3100005091123041950 5    2020  31  ...  3100005091123041950          7

[20 rows x 154 columns]

### Investigando emprego, renda e horas trabalhadas

In [ ]:
## Descrevendo horas normalmente trabalhadas (C008) e de fato trabalhadas na semana passada (C009)
PNAD_COVID.rename(columns={"C008":"horas_hab", "C009":"horas_efet"}, inplace=True)
PNAD_COVID[["horas_hab","horas_efet"]].describe()

# Média caiu, ao mesmo tempo que desvio-padrão aumentou

,horas_hab,horas_efet
count,284387.000000,284387.000000
mean,40.410261,33.302626
std,10.698643,16.864397
min,1.000000,0.000000
25%,40.000000,24.000000
50%,40.000000,40.000000
75%,44.000000,44.000000
max,120.000000,120.000000


In [ ]:
## Descrevendo renda em dinheiro (C01012) ou em produtos (C01022)
PNAD_COVID[["C01012","C01022"]].describe()

,C01012,C01022
count,279265.000000,2181.000000
mean,2482.285918,1048.705640
std,3348.634414,2114.875346
min,6.000000,5.000000
25%,1045.000000,284.000000
50%,1500.000000,420.000000
75%,2500.000000,900.000000
max,200000.000000,20000.000000


In [ ]:
## Somando renda do mês
# Jeito errado (como descobrir que está errado? ver count!):
PNAD_COVID["Salário"] = PNAD_COVID["C01012"] + PNAD_COVID["C01022"]
PNAD_COVID["Salário"].describe()

count     1235.000000
mean      2846.259109
std       2362.557221
min        110.000000
25%       1600.000000
50%       2100.000000
75%       2950.000000
max      20060.000000
Name: Salário, dtype: float64

In [ ]:
# Jeito certo: dois NaNs resultam em um NaN, um valor e outro NaN resulta no valor
PNAD_COVID["Salário"] = PNAD_COVID[["C01012","C01022"]].sum(axis=1, min_count=1)
PNAD_COVID[["Salário", "C01012","C01022"]].describe()

,Salário,C01012,C01022
count,280211.00000,279265.000000,2181.000000
mean,2482.06817,2482.285918,1048.705640
std,3348.12067,3348.634414,2114.875346
min,6.00000,6.000000,5.000000
25%,1045.00000,1045.000000,284.000000
50%,1500.00000,1500.000000,420.000000
75%,2500.00000,2500.000000,900.000000
max,200000.00000,200000.000000,20000.000000


In [ ]:
## Contando números de pessoas que perderam o emprego durante a pandemia
# Criando uma coluna com a contagem de NaNs por grupo
# Usamos o transform para o resultado vir para uma coluna do próprio DataFrame (ao invés de uma série separada com apenas valores do grupo)
PNAD_COVID['quantidade_nas'] = PNAD_COVID.Salário.isnull().groupby('idind').transform('sum').astype(int)

## Vendo o DataFrame
PNAD_COVID['quantidade_nas']

idind                Mês
3100005091012331984  5      0
                     6      0
                     8      0
                     9      0
                     10     0
                           ..
350723620922131965   7      6
                     8      6
                     9      6
                     10     6
                     11     6
Name: quantidade_nas, Length: 693590, dtype: int64

In [ ]:
## quantidade_nas indica o número de vezes que a pessoa não declarou renda, ou seja
## ou estava sem emprego ou estava afastado sem remuneração

## Se esse valor for positivo e for menor que o número de vezes que ela aparece na pesquisa,
## significa que em algum momento ou ela ganhou emprego, ou perdeu, ou foi afastada

PNAD_COVID['perdeu_ganhou_afastado_emprego'] = np.where(
    (PNAD_COVID.quantidade_nas > 0) & (PNAD_COVID.quantidade_nas < PNAD_COVID.Aparições),
    1,
    0
)

# Alternativa: usar DataFrame.apply, mas demora cerca de 100x mais (não é brincadeira)
# Com apply: 2min30s; com np.where, 0.3s

In [ ]:
## Contando
PNAD_COVID['perdeu_ganhou_afastado_emprego'].value_counts(dropna=False, normalize=True)

0    0.868042
1    0.131958
Name: perdeu_ganhou_afastado_emprego, dtype: float64

In [ ]:
## Restringindo a base apenas a quem esteve empregado em algum momento
# (será útil para algumas análises mais a frente)
PNAD_COVID_EMPREGADOS = PNAD_COVID.query('perdeu_ganhou_afastado_emprego == 1 or quantidade_nas == 0')

## Tamanho total da base
print(f"EMPREGADOS\nLinhas: {PNAD_COVID_EMPREGADOS.shape[0]}; Colunas: {PNAD_COVID_EMPREGADOS.shape[1]}")
print(f"TOTAL\nLinhas: {PNAD_COVID.shape[0]}; Colunas: {PNAD_COVID.shape[1]}")

EMPREGADOS
Linhas: 326039; Colunas: 157
TOTAL
Linhas: 693590; Colunas: 157


## Modelos de Painel

### Preparação e Variáveis

In [ ]:
## Criando dummies de cor e gênero
# Feminino
PNAD_COVID['feminino'] = PNAD_COVID['A003'] - 1
PNAD_COVID_EMPREGADOS['feminino'] = PNAD_COVID_EMPREGADOS['A003'] - 1

# Cor
PNAD_COVID.rename(columns={"A004":"cor"}, inplace=True)
PNAD_COVID["cor"].replace([1,2,3,4,5,9], ['a_branca','preta','amarela','parda','indigena', "nao_declarada"], inplace=True)
PNAD_COVID = pd.concat([PNAD_COVID, pd.get_dummies(PNAD_COVID['cor'])], axis=1)

PNAD_COVID_EMPREGADOS.rename(columns={"A004":"cor"}, inplace=True)
PNAD_COVID_EMPREGADOS["cor"].replace([1,2,3,4,5,9], ['a_branca','preta','amarela','parda','indigena',"nao_declarada"], inplace=True)
PNAD_COVID_EMPREGADOS = pd.concat([PNAD_COVID_EMPREGADOS, pd.get_dummies(PNAD_COVID_EMPREGADOS['cor'])], axis=1)

clear_output()

In [ ]:
## Contando valores
# Feminino
PNAD_COVID['feminino'].value_counts(dropna=False, normalize=True)

1    0.524512
0    0.475488
Name: feminino, dtype: float64

In [ ]:
# Cor
PNAD_COVID['cor'].value_counts(dropna=False, normalize=True)

a_branca         0.481496
parda            0.411197
preta            0.098245
amarela          0.007189
indigena         0.001783
nao_declarada    0.000089
Name: cor, dtype: float64

In [ ]:
## Remapeando Educação
Valores_Educ = ["A_SI", "Fund Inc", "Fund Comp", "Méd Inc", "Med Comp", "Sup Inc", "Sup Comp", "Pós"]
PNAD_COVID["Educação"] = PNAD_COVID["A005"].replace([1, 2, 3, 4, 5, 6, 7, 8], Valores_Educ)
PNAD_COVID_EMPREGADOS["Educação"] = PNAD_COVID_EMPREGADOS["A005"].replace([1, 2, 3, 4, 5, 6, 7, 8], Valores_Educ)

In [ ]:
# Log da renda por hora
PNAD_COVID["lsalariohora"] = np.log(PNAD_COVID["Salário"] / (PNAD_COVID["horas_hab"]*4))
PNAD_COVID_EMPREGADOS["lsalariohora"] = np.log(PNAD_COVID_EMPREGADOS["Salário"] / (PNAD_COVID_EMPREGADOS["horas_hab"]*4))

In [ ]:
# Colocando Mês como dummies (e não valores contínuos)
PNAD_COVID["Mês"] = pd.Categorical(PNAD_COVID.Mês)
PNAD_COVID_EMPREGADOS["Mês"] = pd.Categorical(PNAD_COVID_EMPREGADOS.Mês)

In [ ]:
# Imputando valores 0 nos salários nulos (será útil na análise de PNAD_COVID_EMPREGADOS)
PNAD_COVID["lsalariohora0"] = PNAD_COVID["lsalariohora"].fillna(value=0)
PNAD_COVID_EMPREGADOS["lsalariohora0"] = PNAD_COVID_EMPREGADOS["lsalariohora"].fillna(value=0)

### OLS Agrupados

#### Armadilha da Composição

In [ ]:
## Definindo formula
# No linearmodels, precisamos incluir explicitamente um intercepto usando o 1+ (menos quando usarmos primeiras diferenças)
formula_mes = "lsalariohora ~ 1 + Mês"

## Modelo
# Importante: DataFrame tem que estar no modelo MultiIndex indivíduo-tempo!
modelo_mes = PooledOLS.from_formula(formula=formula_mes, data=PNAD_COVID)
modelo_mes = modelo_mes.fit()

## Sumário: note que não há o () ao final
print(modelo_mes.summary)

## Dummies de tempo: salário médio aumentou em julho frente a maio?
# Efeito-Composição: mais afetados pela crise foram os mais vulneráveis;
# como pessoas demitidas tem renda NaN, essas linhas são retiradas da análise, deixando apenas pessoas mais bem-remuneradas

# Mais detalhes: https://blogdoibre.fgv.br/posts/decomposicao-mostra-que-impacto-da-pandemia-influenciou-renda-media-positivamente

/usr/local/lib/python3.7/dist-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PooledOLS Estimation Summary                          
Dep. Variable:           lsalariohora   R-squared:                     6.613e-05
Estimator:                  PooledOLS   R-squared (Between):             -0.0033
No. Observations:              280211   R-squared (Within):               0.0002
Date:                Fri, Oct 22 2021   R-squared (Overall):           6.613e-05
Time:                        00:44:11   Log-likelihood                -3.305e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.0885
Entities:                       49602   P-value                           0.0050
Avg Obs:                       5.6492   Distribution:                F(6,280204)
Min Obs:                       1.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             3.0885
                            

In [ ]:
## Imputando os 0s como proxy para coletar pessoas que perderam o emprego
formula_mes_0 = "lsalariohora0 ~ 1 + Mês"

## Modelo
modelo_mes_0 = PooledOLS.from_formula(formula=formula_mes_0, data=PNAD_COVID_EMPREGADOS)
modelo_mes_0 = modelo_mes_0.fit()
print(modelo_mes_0.summary)

## Note que, ao considerar apenas pessoas que tinham emprego em algum momento (podendo tê-lo perdido, 
# o que é captado pelo 0 no lugar do NaN), houve perda de renda na pandemia frente a maio (junho a setembro)
# Não se pode usar lsalario0 com a base toda em virtude de existirem pessoas nunca empregadas

                          PooledOLS Estimation Summary                          
Dep. Variable:          lsalariohora0   R-squared:                        0.0004
Estimator:                  PooledOLS   R-squared (Between):          -5.142e-05
No. Observations:              326039   R-squared (Within):               0.0014
Date:                Fri, Oct 22 2021   R-squared (Overall):              0.0004
Time:                        00:47:18   Log-likelihood                -4.962e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      19.399
Entities:                      105605   P-value                           0.0000
Avg Obs:                       3.0873   Distribution:                F(6,326032)
Min Obs:                       0.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             19.399
                            

#### Modelos

In [ ]:
## Definindo formula
# No linearmodels, precisamos incluir explicitamente um intercepto usando o 1+ (menos quando usarmos primeiras diferenças)
formula = "lsalariohora0 ~ 1 + Mês + C(Educação) + feminino + C(cor)"

## Modelo
modelo_olsagrup = PooledOLS.from_formula(formula=formula, data=PNAD_COVID_EMPREGADOS)
modelo_olsagrup = modelo_olsagrup.fit(cov_type="robust")  # controlando para heteroscedasticidade

In [ ]:
print(modelo_olsagrup.summary)

                          PooledOLS Estimation Summary                          
Dep. Variable:          lsalariohora0   R-squared:                        0.2083
Estimator:                  PooledOLS   R-squared (Between):              0.2836
No. Observations:              326039   R-squared (Within):              -0.0015
Date:                Fri, Oct 22 2021   R-squared (Overall):              0.2083
Time:                        00:48:57   Log-likelihood                -4.582e+05
Cov. Estimator:                Robust                                           
                                        F-statistic:                      4514.7
Entities:                      105605   P-value                           0.0000
Avg Obs:                       3.0873   Distribution:               F(19,326019)
Min Obs:                       0.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             3960.3
                            

### Efeitos Fixos

In [ ]:
## Criando modelo
# Definindo fórmula
formula_ef = f"{formula} + EntityEffects"

# Criando o modelo
# drop_absorbed: se tiver alguma variável constante no tempo, pulá-la
modelo_ef = PanelOLS.from_formula(formula=formula_ef, data=PNAD_COVID_EMPREGADOS, drop_absorbed=True)
modelo_ef = modelo_ef.fit(cov_type='robust')

# Printando resultado
print(modelo_ef.summary)

# feminino é estatisticamente insignificante por conta da baixa variação da amostra
# (ou erro de preenchimento ou pessoas que mudaram sua identificação de gênero)

# R2: "porcentagem da variação temporal em y explicada pela variação temporal em X"

                          PanelOLS Estimation Summary                           
Dep. Variable:          lsalariohora0   R-squared:                        0.0030
Estimator:                   PanelOLS   R-squared (Between):              0.0801
No. Observations:              326039   R-squared (Within):               0.0030
Date:                Fri, Oct 22 2021   R-squared (Overall):              0.0604
Time:                        00:52:12   Log-likelihood                -2.795e+05
Cov. Estimator:                Robust                                           
                                        F-statistic:                      43.051
Entities:                      105605   P-value                           0.0000
Avg Obs:                       3.0873   Distribution:               F(19,276418)
Min Obs:                       0.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             27.781
                            

In [ ]:
## Discriminacao feminina teve alguma mudança ao longo do tempo?
# Interação entre variáveis!
formula_ef_interacao = "lsalariohora0 ~ 1 + Mês + feminino + feminino:Mês + C(Educação) + C(cor) + EntityEffects"

## Modelo
modelo_ef_interacao = PanelOLS.from_formula(formula=formula_ef_interacao, data=PNAD_COVID_EMPREGADOS, drop_absorbed=True)
modelo_ef_interacao = modelo_ef_interacao.fit(cov_type='robust')

# Printando resultado
print(modelo_ef_interacao.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:          lsalariohora0   R-squared:                        0.0037
Estimator:                   PanelOLS   R-squared (Between):              0.0791
No. Observations:              326039   R-squared (Within):               0.0037
Date:                Fri, Oct 22 2021   R-squared (Overall):              0.0599
Time:                        00:54:00   Log-likelihood                -2.793e+05
Cov. Estimator:                Robust                                           
                                        F-statistic:                      40.970
Entities:                      105605   P-value                           0.0000
Avg Obs:                       3.0873   Distribution:               F(25,276412)
Min Obs:                       0.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             26.632
                            

### Efeitos Aleatórios

In [ ]:
## Criando modelo
# Definindo fórmula
formula_ea = "lsalariohora0 ~ 1 + Mês + feminino + C(Educação) + C(cor)"

# Criando o modelo
modelo_ea = RandomEffects.from_formula(formula=formula_ea, data=PNAD_COVID_EMPREGADOS)
modelo_ea = modelo_ea.fit(cov_type='robust')

# Printando resultado
print(modelo_ea.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:          lsalariohora0   R-squared:                        0.0574
Estimator:              RandomEffects   R-squared (Between):              0.2811
No. Observations:              326039   R-squared (Within):              -0.0003
Date:                Fri, Oct 22 2021   R-squared (Overall):              0.2068
Time:                        00:55:46   Log-likelihood                -3.069e+05
Cov. Estimator:                Robust                                           
                                        F-statistic:                      1044.0
Entities:                      105605   P-value                           0.0000
Avg Obs:                       3.0873   Distribution:               F(19,326019)
Min Obs:                       0.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             838.85
                            

In [ ]:
## Criando modelo (consome MUITA memória e o Colab dá erro)
# Definindo fórmula
formula_ea_interacao = "lsalariohora0 ~ 1 + Mês + feminino + feminino:Mês + C(Educação) + C(cor)"

# Criando o modelo
modelo_ea_interacao = RandomEffects.from_formula(formula=formula_ea_interacao, data=PNAD_COVID_EMPREGADOS)
modelo_ea_interacao = modelo_ea_interacao.fit(cov_type='robust')

# Printando resultado
print(modelo_ea_interacao.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:          lsalariohora0   R-squared:                        0.0579
Estimator:              RandomEffects   R-squared (Between):              0.2811
No. Observations:              326039   R-squared (Within):               0.0005
Date:                Fri, Oct 22 2021   R-squared (Overall):              0.2070
Time:                        00:56:23   Log-likelihood                -3.068e+05
Cov. Estimator:                Robust                                           
                                        F-statistic:                      801.98
Entities:                      105605   P-value                           0.0000
Avg Obs:                       3.0873   Distribution:               F(25,326013)
Min Obs:                       0.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             649.17
                            

### Teste de Hausman

Ideia: comparar os resultados de efeitos fixos e efeitos aleatórios.
Se forem muito diferentes, é sinal que há correlação da variável independente com a característica não-observável, o que torna EA inconsistente.
Caso contrário, EF ainda é consistente, mas EA é mais eficiente (possui menores erros-padrão dos estimadores, resultando em estimativas mais precisas)

In [ ]:
##### Modelo sem interação
## Calculando estatística de Hausman
# Variância Assintótica
var_assin = modelo_ef.cov - modelo_ea.cov

# Diferença entre os parâmetros
dif_parametros = modelo_ef.params - modelo_ea.params

# Calculando a estatística de Hausman
H = dif_parametros.dot(np.linalg.inv(var_assin)).dot(dif_parametros)

# Grau de Liberdade
graus_liberdade = modelo_ea.params.size - 1

# Calculando p-valor
# H0: não há correlação entre as variáveis independentes e as características não-observáveis
# Não-rejeitar H0: prefere-se EA, mas EF ainda é consisteente
# Rejeitar: prefere-se EF e EA é inconsistente
p = stats.chi2(graus_liberdade).sf(H)

print(p)

1.38821855520985e-69


In [ ]:
##### Modelo com interação
## Calculando estatística de Hausman
# Variância Assintótica
var_assin = modelo_ef_interacao.cov - modelo_ea_interacao.cov

# Diferença entre os parâmetros
dif_parametros = modelo_ef_interacao.params - modelo_ea_interacao.params

# Calculando a estatística de Hausman
H = dif_parametros.dot(np.linalg.inv(var_assin)).dot(dif_parametros)

# Grau de Liberdade
graus_liberdade = modelo_ea_interacao.params.size - 1

# Calculando p-valor
# H0: não há correlação entre as variáveis independentes e as características não-observáveis
# Não-rejeitar H0: prefere-se EA, mas EF ainda é consisteente
# Rejeitar: prefere-se EF e EA é inconsistente
p = stats.chi2(graus_liberdade).sf(H)

print(p)

4.8229138544701644e-64
